In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [92]:
#loads pretrained model for vectorizing sentences, Word2Vec
path = "GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(path, binary=True, limit = 2000)
vocab_size = len(model.index_to_key)
vocab_size

2000

In [46]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
def embedSentence(row):
    #uses nltk to seperate sentence into words omitting punctuation
    tokens = tokenizer.tokenize(row)
    #averages all token vectors
    vector = np.array([0]*300)
    count = 0
###################Need to fix bug with possible zero count
    for i in tokens:
        if i in model: #only adds tokens present in the model
            vector = vector + model.get_vector(i)
            count += 1
    vector = vector/max(1, count)
    return vector

In [82]:
#loads emotion training csv
df = pd.read_csv("emotions.csv")
df

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like telling these horny devils to find...,2
416805,i began to realize that when i was feeling agi...,3
416806,i feel very curious be why previous early dawn...,5
416807,i feel that becuase of the tyranical nature of...,3


In [102]:
#embeds all x from word to values
df["text"] = df["text"].apply(embedSentence)


In [103]:
X = df["text"].values
y = df["label"].values

In [104]:
X = np.stack(X).astype(np.float32)

In [105]:
#split x, y into training
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=0)

In [106]:
#builds tensorflow model
tf_model = tf.keras.Sequential([
                                tf.keras.layers.Dense(64, activation='relu'),
                                tf.keras.layers.Dense(6, activation='softmax')
                                ])

In [107]:
tf_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=["accuracy"])

In [108]:
tf_model.evaluate(X_train, y_train)

7816/7816 ━━━━━━━━━━━━━━━━━━━━ 7s 771us/step - accuracy: 0.2933 - loss: 1.7487


[1.748745083808899, 0.2941639721393585]

In [109]:
tf_model.fit(X_train, y_train, batch_size=16, epochs = 5, validation_data=(X_valid, y_valid))

Epoch 1/5
15631/15631 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.4028 - loss: 1.5008 - val_accuracy: 0.4088 - val_loss: 1.4713
Epoch 2/5
15631/15631 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.4242 - loss: 1.4558 - val_accuracy: 0.4230 - val_loss: 1.4541
Epoch 3/5
15631/15631 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.4272 - loss: 1.4487 - val_accuracy: 0.4274 - val_loss: 1.4443
Epoch 4/5
15631/15631 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - accuracy: 0.4321 - loss: 1.4421 - val_accuracy: 0.4237 - val_loss: 1.4478
Epoch 5/5
15631/15631 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.4345 - loss: 1.4336 - val_accuracy: 0.4298 - val_loss: 1.4407
